# Torsionsweicher Trägerrost

Das Anwendungbeispiel des torsionsweichen Trägerrosts glieder sich in den Aufgabenbeschrieb, der analytischen Lösung und der Modellierung mit dem NLFE-Modell inklusiven dessen Resultaten. Das Beispiel soll die Anwendbarkeit an dreidimensionalen Stabtragwerken aufzeigen.

## Aufgabenbeschrieb

Das Beispiel stammt aus @marti_baustatik_2014. In der @fig-trm_uebersicht ist der Grundriss des Trägerrosts dargestellt. Es handelt sich um insgesamt 16 torsionsweiche Stäbe, welche an den Enden eingespannt sind.

![Grundriss des torsionsweichen Trägerrosts, neugezeichnet nach @marti_baustatik_2014](../imgs/trm_uebersicht.svg){#fig-trm_uebersicht}

Es ist die Traglast des Systems zu ermitteln. Die Stablänge und die positiven und negativen Biegewiderstände sind die folgenden.

In [1]:
# | echo: false

from packages import *

In [2]:
# Traglast

# Rastergrösse
l = 1 * un.m  




# Positiver Biegewiderstand
M_u = 100 * un.kN * un.m  

# Negativer Biegewiderstand
M_apos_u = 100 * un.kN * un.m

put_out()

$$
\begin{aligned}
l& = 1 \ \mathrm{m} \quad & M_{u}& = 100 \ \mathrm{kN} \cdot \mathrm{m} \quad & {M}'_{u}& = 100 \ \mathrm{kN} \cdot \mathrm{m} \end{aligned}
$$

## Analytische Lösung

Die Analytische Lösung basiert auf dem Traglastverfahren, beschrieben in @marti_baustatik_2014. Mittels einem zulässigen Mechanismus ist ein oberer Grenzwert der Traglast herzuleiten. In der @fig-trm_schnitt sind zwei Stäbe dargestellt. Aus Symmetriegründen lässt sich der obere Grenzwert des gesamten Trägerrosts anhand dieser Darstellung ermitteln.

![Schnitt des torsionsweichen Trägerrosts, mit dem angenommenen Mechanismus für den Innen- und Aussenträger](../imgs/trm_schnitt.svg){#fig-trm_schnitt}



Die äussere Arbeit $W_a$ des dargestellten Systems in @fig-trm_schnitt beträgt dabei für die am Rand gelegenen Stäbe (Punkte 2'45):

$$
W_{a,2'45} = 4 \cdot \left( Q \cdot \frac{1}{3} + Q \cdot \frac{1}{9} \right)
$$

Und für die Innenträger:

$$
W_{a,123} = 4 \cdot \left( Q \cdot 1 + Q \cdot \frac{1}{3} \right)
$$


Sowie beträgt die innere Arbeit:

$$
W_i = 8 \cdot \left(M_u' + M_u\right) \cdot \left(\frac{1}{3l} +\frac{1}{9l}\right)
$$

Durch das abschliessende Gleichsetzen der Arbeiten und das Lösen nach $Q$ folgt der obere Grenzwert der Traglast zu:

$$
Q = \frac{M_u + M_u'}{2 \cdot l}
$$

Die Plastizitätskontrolle ist in der @fig-trm_schnitt_my und @fig-trm_schnitt_vz gezeigt. Dabei wird eine Lastverteilung von je einer Hälfte $Q$ in $x$ und $y$ Richtung angenommen. Welche sich nach vollständigen Umlagern der Biegemomente einstellt.

![Plastizitätskontrolle anhand der Schnittgrössen der Biegemomente](../imgs/trm_schnitt_my.svg){#fig-trm_schnitt_my}

![Plastizitätskontrolle anhand der Schnittgrössen der Querkräfte](../imgs/trm_schnitt_vz.svg){#fig-trm_schnitt_vz}

Aus der Plastizitätskontrolle geht heraus, dass der Biegewiderstand nirgends überschritten ist, somit deckt sich der untere und obere Grenzwert der Traglast, sprich die Traglast $Q_u$ ist gefunden.


$$
Q_u = \frac{M_u + M_u'}{2 \cdot l}
$$



Mit den numerischen Grössen gemäss der Aufgabenstellung entspricht die Traglast dem Folgenden.

In [3]:
Q_u = (M_u + M_apos_u) / (2*l)

put_out(symbolic=True, rows=1)

$$
\begin{aligned}
Q_{u}& = \frac{M_{u} + {M}'_{u}}{2 \cdot l} = 100.0 \ \mathrm{kN} \end{aligned}
$$

## NLFE-Modell


Die analytische Lösung ist mit dem NLFE-Modell zu nähern. Dazu ist die Systemmodellbildung beschrieben und die Querschnittsmodellbildung. Abschliessend sind die Resultate aufgezeigt. 

### Systemmodellbildung

Das System ist in biegestarre Stäbe unterteilt und mit den Gelenken versehen. Die @fig-trm_geom zeigt das statische System. Die Einwirkung ist mit 120 kN modelliert.

![Statisches Modell in AxisVM des Trägerrosts](../imgs/trm_system.svg){#fig-trm_geom}

Die Gelenke und die Elementlänge ist in der detaillierten @fig-trm_detail_system zu sehen.



![Detailausschnitt des Trägerrosts mit Gelenken und vermasster Elementlänge](../imgs/trm_detail_system.svg){#fig-trm_detail_system}

### Querschnittsmodellbildung

#### Biegesteifigkeit

Bei der Ermittlung der Traglast nach dem Traglastverfahren spielt die Verformung des Systems keine Rolle. Lediglich der Biegewiderstand ist von Interesse. Das Verhalten lässt sich mit einem ideal-plastischen Momenten-Krümmungs-Diagramm beschreiben. Aufgezeigt in der @fig-trm_m_chi.

In [4]:
# | echo: false

radians = np.linspace(-0.2, 0.2, 100)
M = np.piecewise(radians, [radians < 0, radians > 0], [-100, 100])

fig, ax = plt.subplots()

ax.plot(radians, M, linestyle="-", color="k")
ax.plot(radians, radians * 0, color="gray")

# Set axis labels
ax.set_xlabel(r"$\varphi$ [rad]")
ax.set_ylabel(r"$M$ [kNm]")

# Set limits
# ax.set_xlim([-0.2, 0.2])
# ax.set_ylim([-101, 101])

# Show the plot
plt.tight_layout()
# plt.savefig("imgs/trm_gelenkdef.svg")
plt.close()

![Ideal-plastische a) Momenten-Krümmungs-Beziehung, b) Momenten-Verdrehungs-Beziehung ](../imgs/trm_m_chi.svg){#fig-trm_m_chi}


Das ideal-plastische Verhalten ist punktsymmetrisch, gilt somit ebenfalls für negative Biegemomente.

#### Abbruchkriterium

Bei der Traglastermittlung des Systems ist das Rotationsvermögen des Systems von keiner bedeutung. Die Traglast stellt sich bei einer vollständigen Umlagerung sämtlicher Biegemomente ein. Die Traglast ist gefunden, wenn sich im Last-Verformungs-Diagramm ein asymptotischer Verlauf einstellt.

### Resultate

Die folgenden Abbildungen zeigen die Verteilung der Biegemomente und der Querkräfte im Trägerrost, ermittelt mit dem NLFE-Modell. Die ersten Gelenke bei den innenliegenden Stäbe bilden sich bei folgender Last.

Bei den Aussenträgern bilden sich plastische Gelenke bei folgender Laststufe.

In [7]:
Q_2 = Q * 0.758

put_out(symbolic=True)

$$
\begin{aligned}
Q_{2}& = Q \cdot 0.758 = 90.96 \ \mathrm{kN} \quad &  \quad &  
 \end{aligned}
$$

Die folgenden beiden Abbildungen zeigen die Zustandslinien der Biegemomente und der Querkräfte. Der Biegewiderstand ist bei sämtlichen Auflagernpunkten erreicht.

Die Bildung des Fliessgelenks im Feld bei den Innenträgern liegt nahe bei der vorherigen Laststufe.


In [8]:
Q_3 = Q*0.781

put_out(symbolic=True)

$$
\begin{aligned}
Q_{3}& = Q \cdot 0.781 = 93.72 \ \mathrm{kN} \quad &  \quad &  
 \end{aligned}
$$

Die Zustandslinien dazu sind folgend gezeigt. Es zeigt sich, dass in Feldmitte der Biegewiderstand erreicht ist.

Abschliessend gibt es ein Konvergenzproblem bei folgender Laststufe.

In [9]:
Q_4 = Q *  0.832

put_out(symbolic=True)

$$
\begin{aligned}
Q_{4}& = Q \cdot 0.832 = 99.84 \ \mathrm{kN} \quad &  \quad &  
 \end{aligned}
$$

Die Zustandslinien der Biegemomente zeigen ein Erreichen des Biegewiderstands im Feld der Aussenträger. Die Traglast ist erreicht.

## Fazit

-